# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,18.41,71,90,5.40,SH,1707406926
1,1,pizarro,4.9533,-77.3660,28.77,71,66,1.00,CO,1707406927
2,2,margaret river,-33.9500,115.0667,21.67,46,0,5.14,AU,1707406928
3,3,jaru,-10.4389,-62.4664,29.72,72,100,1.52,BR,1707406652
4,4,sylt-ost,54.8833,8.3500,3.05,81,100,1.54,DE,1707406929


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
cities_hum = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = 'Humidity' ,
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_hum


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria
criteria_df = city_data_df[(city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
my_cities = criteria_df.dropna(how='any')

# Display the filtered DataFrame
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,margaret river,-33.9500,115.0667,21.67,46,0,5.14,AU,1707406928
36,36,bereket,39.2446,55.5154,15.34,33,0,1.89,TM,1707406950
55,55,lubbock,33.6001,-101.8338,9.65,49,20,7.20,US,1707406839
82,82,broken hill,-31.9500,141.4333,23.56,36,58,4.59,AU,1707406974
87,87,bethel,41.3712,-73.4140,8.13,49,0,0.45,US,1707406758


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] =''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,margaret river,AU,-33.9500,115.0667,46,
36,bereket,TM,39.2446,55.5154,33,
55,lubbock,US,33.6001,-101.8338,49,
82,broken hill,AU,-31.9500,141.4333,36,
87,bethel,US,41.3712,-73.4140,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1



params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = my_cities.loc[index, 'Lat']
    longitude = my_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
bereket - nearest hotel: Şemşadyñ mamasy
lubbock - nearest hotel: DoubleTree by Hilton Lubbock University Area
broken hill - nearest hotel: Ibis Styles
bethel - nearest hotel: Hampton Inn Danbury
roosevelt - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
tegueste - nearest hotel: Hotel Aguere
barkhan - nearest hotel: No hotel found
ar rutbah - nearest hotel: No hotel found
mandsaur - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
tamanrasset - nearest hotel: فندق الأمان
awjilah - nearest hotel: No hotel found
najran - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
boudjima - nearest hotel: Hôtel Mizrana
tura - nearest hotel: No hotel found
sarina - nearest hotel: Tandara Hotel
zabol - nearest hotel: No hotel found
tucumcari - nearest hotel: Pow Wow Inn
hun - nearest hotel: فندق الهروج
carahue -

,City,Country,Lat,Lng,Humidity,Hotel Name
2,margaret river,AU,-33.9500,115.0667,46,Margaret River Hotel
36,bereket,TM,39.2446,55.5154,33,Şemşadyñ mamasy
55,lubbock,US,33.6001,-101.8338,49,DoubleTree by Hilton Lubbock University Area
82,broken hill,AU,-31.9500,141.4333,36,Ibis Styles
87,bethel,US,41.3712,-73.4140,49,Hampton Inn Danbury
96,roosevelt,US,33.9668,-103.4505,43,No hotel found
99,hamilton,US,39.1834,-84.5333,46,North Vista Manor
108,tegueste,ES,28.5167,-16.3167,42,Hotel Aguere
115,barkhan,PK,29.8977,69.5256,27,No hotel found
116,ar rutbah,IQ,33.0381,40.2848,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    scale = 3,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
map_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)